In [1]:
using Pipe
using Iterators
using HDF5, JLD

In [2]:
using PyCall
@pyimport warnings
warnings.filterwarnings("ignore")

In [3]:
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport nltk_tree_corp_load

In [4]:
@pyimport nltk.corpus as nltk_corpus


In [5]:
function tupup(x::String)
    x
end

function tupup(xs::Array)
    if length(xs)==1
        tupup(xs[1])
    else
        sd = findfirst([size(xs)...], 2)
        
        left = slicedim(xs,sd,1)
        right = slicedim(xs,sd,2)
        (tupup(left), tupup(right))
    end
end

tupup (generic function with 2 methods)

In [6]:
function loadtrees(reader::PyObject)
    @pipe (
    nltk_tree_corp_load.load_treebank_cnf_sents(reader)
    |> imap(tupup,_) 
    |> filter(x->typeof(x)!==UTF8String,_) 
    |> collect)
end

loadtrees (generic function with 1 method)

In [8]:
open("training_sents.jsz","w") do fs
    serialize(fs, loadtrees(nltk_corpus.treebank))
end

In [9]:
@pyimport glob
opinosis_filenames = glob.glob("../Corpus/corpora/Opinosis/processed/topics/*.parsed.txt");
opinosis_reader = nltk_corpus.BracketParseCorpusReader(".", opinosis_filenames)

open("opinosis_sents.jsz","w") do fs
    serialize(fs, loadtrees(opinosis_reader))
end

Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered by returning a flat parse.
Bad tree detected; trying to recover...
  Recovered 